#  TP2 - Clustering (Sans Réduction de Dimension)

 Ce notebook charge les données Hi-Seq normalisées (`hiseq_X_scaled.npy`) et applique les algorithmes K-Means, DBSCAN et Clustering Spectral sur l'ensemble des attributs.

In [4]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import time
import tracemalloc
import warnings

In [9]:
# Chemin racine du projet (le dossier parent de ce notebook)
PROJECT_ROOT = Path("..").resolve()

# Chemin des données traitées (en entrée pour CE notebook)
DATA_PROCESSED_PATH = PROJECT_ROOT / 'data' / 'HISEQ'

# Ajouter le dossier /src au path pour les modules .py
PROJECT_FCT = PROJECT_ROOT / "src"
sys.path.append(str(PROJECT_FCT))

print(f"Chemin du projet racine : {PROJECT_ROOT}")
print(f"Chargement des données depuis : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Chargement des données depuis : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data\HISEQ


In [ ]:
try:
    from utils import evaluate_clustering
    print("Modules locaux 'utils' chargés avec succès.")
except ImportError:
    print("Attention: Les fichiers utils.py (evaluate_clustering, plot_clusters) n'ont pas été trouvés dans /src.")

warnings.filterwarnings('ignore')

Modules locaux 'utils' chargés avec succès.


In [11]:
print("Chargement des données Hi-Seq normalisées...")
try:
    X_scaled = np.load(DATA_PROCESSED_PATH / 'hiseq_X_scaled.npy')
    y_hiseq = pd.read_csv(DATA_PROCESSED_PATH / 'hiseq_y_labels.csv', index_col=0).iloc[:, 0]
    print(f"Données chargées. Forme de X_scaled: {X_scaled.shape}")
except FileNotFoundError:
    print(f"ERREUR: Fichiers non trouvés dans {DATA_PROCESSED_PATH}")
    print("Veuillez d'abord exécuter le notebook 'PreparationData.ipynb'.")

Chargement des données Hi-Seq normalisées...
Données chargées. Forme de X_scaled: (801, 20531)


#  1. K-Means

In [8]:
print("Exécution de K-Means (Full)...")
kmeans_full = KMeans(n_clusters=5, random_state=42, n_init=10)
labels_kmeans_full = kmeans_full.fit_predict(X_scaled)
results_kmeans_full = evaluate_clustering(X_scaled, labels_kmeans_full, y_hiseq)
print(results_kmeans_full)

Exécution de K-Means (Full)...


NameError: name 'X_scaled' is not defined

# 2. DBSCAN 

In [ ]:
# --- Trouver EPS pour DBSCAN (Full) ---
print("Calcul de EPS pour DBSCAN...")
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)
distances = np.sort(distances, axis=0)[:, 4]
kneedle = KneeLocator(range(len(distances)), distances, curve='convex', direction='increasing')
EPS_FULL_OPTIMAL = kneedle.elbow_y if kneedle.elbow_y else 170 # Fallback
print(f"EPS Optimal trouvé: {EPS_FULL_OPTIMAL}")

Calcul de EPS pour DBSCAN (Full)...
EPS Optimal (Full) trouvé: 172.66466987135783


In [ ]:
print("Exécution de DBSCAN...")
dbscan_full = DBSCAN(eps=EPS_FULL_OPTIMAL, min_samples=5)
labels_dbscan_full = dbscan_full.fit_predict(X_scaled)
results_dbscan_full = evaluate_clustering(X_scaled, labels_dbscan_full, y_hiseq, allow_noise=True)
print(results_dbscan_full)

Exécution de DBSCAN (Full)...
{'Silhouette': nan, 'Homogeneity': 0.0, 'Completeness': 1.0, 'V-Measure': 0.0, 'ARI': 0.0, 'Noise_Percentage': 5.742821473158552}


# 3. Clustering Spectral

In [ ]:
print("Exécution de Spectral Clustering...")
# Attention: L'exécution sur 20k features peut être très longue ou échouer.
try:
    spectral_full = SpectralClustering(n_clusters=5, assign_labels='kmeans', random_state=42)
    labels_spectral_full = spectral_full.fit_predict(X_scaled)
    results_spectral_full = evaluate_clustering(X_scaled, labels_spectral_full, y_hiseq)
    print(results_spectral_full)
except Exception as e:
    print(f"Erreur lors du clustering spectral: {e}")
    print("Cette méthode est très coûteuse sur la dimension complète.")

Exécution de Spectral Clustering (Full)...
{'Silhouette': -0.0589012149481846, 'Homogeneity': 0.004904285780407865, 'Completeness': 0.010567042378870902, 'V-Measure': 0.006699333780026259, 'ARI': 0.003306452428438863}


# 4. Comparaison

In [ ]:
results_df = pd.DataFrame([results_kmeans_full, results_dbscan_full, results_spectral_full])
results_df.index = ['KMeans (Full)', 'DBSCAN (Full)', 'Spectral (Full)']
print(results_df.to_markdown(floatfmt=".4f"))

|                 |   Silhouette |   Homogeneity |   Completeness |   V-Measure |    ARI |   Noise_Percentage |
|:----------------|-------------:|--------------:|---------------:|------------:|-------:|-------------------:|
| KMeans (Full)   |       0.1353 |        0.8632 |         0.8493 |      0.8562 | 0.8001 |           nan      |
| DBSCAN (Full)   |     nan      |        0.0000 |         1.0000 |      0.0000 | 0.0000 |             5.7428 |
| Spectral (Full) |      -0.0589 |        0.0049 |         0.0106 |      0.0067 | 0.0033 |           nan      |
